In [17]:
%load_ext autoreload
%autoreload 2

import pickle
import pandas as pd
import numpy as np

import sys
sys.path.append("../")
from src import soporte_eda_y_outliers as s_eda, soporte_encoding as s_encoding, soporte_scaling as s_scaling


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## En el modelo anterior tuvimos dos principales problemas, el overfitting general y crear un encoding más claro. Intentaremos paliar esos problemas empezando por el EDA.

In [18]:
df_survey = pd.read_csv("../datos_origen/employee_survey_data.csv")
df_general = pd.read_csv("../datos_origen/general_data.csv")
df_manager = pd.read_csv("../datos_origen/manager_survey_data.csv")

In [19]:
df_a = df_survey.merge(df_manager, on= "EmployeeID", how = "inner")
df = df_a.merge(df_general, on= "EmployeeID", how = "inner")
df

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,BusinessTravel,Department,...,NumCompaniesWorked,Over18,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
0,1,3.0,4.0,2.0,3,3,51,No,Travel_Rarely,Sales,...,1.0,Y,11,8,0,1.0,6,1,0,0
1,2,3.0,2.0,4.0,2,4,31,Yes,Travel_Frequently,Research & Development,...,0.0,Y,23,8,1,6.0,3,5,1,4
2,3,2.0,2.0,1.0,3,3,32,No,Travel_Frequently,Research & Development,...,1.0,Y,15,8,3,5.0,2,5,0,3
3,4,4.0,4.0,3.0,2,3,38,No,Non-Travel,Research & Development,...,3.0,Y,11,8,3,13.0,5,8,7,5
4,5,4.0,1.0,3.0,3,3,32,No,Travel_Rarely,Research & Development,...,4.0,Y,12,8,2,9.0,2,6,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4405,4406,4.0,1.0,3.0,3,3,42,No,Travel_Rarely,Research & Development,...,3.0,Y,17,8,1,10.0,5,3,0,2
4406,4407,4.0,4.0,3.0,2,3,29,No,Travel_Rarely,Research & Development,...,2.0,Y,15,8,0,10.0,2,3,0,2
4407,4408,1.0,3.0,3.0,3,4,25,No,Travel_Rarely,Research & Development,...,0.0,Y,20,8,0,5.0,4,4,1,2
4408,4409,4.0,1.0,3.0,2,3,42,No,Travel_Rarely,Sales,...,0.0,Y,14,8,1,10.0,2,9,7,8


In [20]:
df = df.drop(columns = ["Over18","EmployeeCount","StandardHours"])
df["Attrition"] = df["Attrition"].map({"Yes": 1, "No": 0})

El análisis original era correcto. Ahora vamos a eliminar otras variables en el preproceso, con intención de buscar algo más de información contextual. En primer lugar eliminaremos "PerformanceRating" frente a "PercentSalaryHike" porque en nuestro modelo el sueldo tenía más importancia que la performance. Aunque sueldo y el porcentaje de subida no parecen estar relacionados puede aportar valor extra. Eliminaremos "Yearswithcurrentmanager" frente a "Yearsatcompany", pues aunque hayamos visto que esuna variable muy importante en el modelo, ya hemos contestado a la pregunta anterior y queremos ver si tiene que ver con el efecto de los años en la empresa. Haremos lo mismo con "Age" y "YearsSinceLastPromotion"

In [21]:
visual = s_eda.Visualizador(df)
visual.dataframe_correlación()

,EmployeeID,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating,Age,Attrition,DistanceFromHome,Education,JobLevel,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
EmployeeID,1.000000,0.000798,-0.001053,-0.001723,0.008591,-0.006892,0.008649,-0.004729,-0.001097,-0.009680,-0.003303,0.007338,-0.001117,-0.004456,-0.014254,-0.001063,-0.010191,0.004086,0.000256,0.008579
EnvironmentSatisfaction,0.000798,1.000000,-0.006550,0.024379,0.015416,0.014057,0.010089,-0.102142,0.020183,-0.045469,-0.024448,-0.006006,0.013007,0.000841,-0.004591,-0.002265,0.018003,0.000421,0.015283,-0.005876
JobSatisfaction,-0.001053,-0.006550,1.000000,-0.019104,0.004058,0.043261,-0.003086,-0.102847,-0.011144,-0.004173,-0.018279,0.005063,-0.056230,0.030808,0.039907,-0.016849,-0.021514,-0.002166,-0.016236,-0.024612
WorkLifeBalance,-0.001723,0.024379,-0.019104,1.000000,-0.019424,-0.023067,-0.021297,-0.062756,0.008000,-0.006406,-0.027664,0.003572,-0.008714,-0.040601,-0.022886,0.001119,-0.015817,0.014704,0.009234,0.007456
JobInvolvement,0.008591,0.015416,0.004058,-0.019424,1.000000,0.010699,0.018196,-0.015588,-0.001837,-0.018279,-0.012630,0.023346,0.027985,-0.004658,0.010295,0.006709,-0.010144,0.008314,0.025395,-0.001372
PerformanceRating,-0.006892,0.014057,0.043261,-0.023067,0.010699,1.000000,-0.025563,0.023403,0.036418,-0.035591,-0.002474,0.016093,0.017819,0.773550,-0.036348,-0.001396,-0.022895,-0.011038,-0.020159,-0.009426
Age,0.008649,0.010089,-0.003086,-0.021297,0.018196,-0.025563,1.000000,-0.159205,0.006963,-0.035706,-0.002884,-0.044314,0.299243,-0.033137,-0.031753,0.680661,-0.027308,0.311309,0.216513,0.202089
Attrition,-0.004729,-0.102142,-0.102847,-0.062756,-0.015588,0.023403,-0.159205,1.000000,-0.009730,-0.015111,-0.010290,-0.031176,0.042345,0.032533,-0.006839,-0.170338,-0.049431,-0.134392,-0.033019,-0.156199
DistanceFromHome,-0.001097,0.020183,-0.011144,0.008000,-0.001837,0.036418,0.006963,-0.009730,1.000000,-0.008638,-0.037329,-0.021607,-0.013843,0.038125,0.011169,0.009374,-0.009001,0.031684,0.002290,0.021584
Education,-0.009680,-0.045469,-0.004173,-0.006406,-0.018279,-0.035591,-0.035706,-0.015111,-0.008638,1.000000,0.045746,0.006410,-0.016250,-0.040531,0.001261,-0.010717,0.010472,0.006080,0.022490,0.005358


In [22]:
df.drop(columns = ["PerformanceRating","YearsWithCurrManager","TotalWorkingYears","YearsSinceLastPromotion"],inplace = True)

Aquí no tocaremos los tipos de cambio del salario, esperamos que un menor tipo de cambio nos de mayor capacidad de análisis fino

Guardamos y probamos :)


In [23]:
df.to_pickle("../datos_dfs/pickle2_preprocesado.pkl")